## IMPORTING ALL THE MODELS WHICH ARE REQUIRED

In [2]:
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import cv2

In [3]:
import os
import random as rd
import glob
import numpy as np
import cv2

## Reading the data

In [4]:
Arborio = glob.glob(r"C:\Users\HP\Desktop\data sets\Rice_Image_Dataset\Arborio/*.*")
Basmati = glob.glob(r"C:\Users\HP\Desktop\data sets\Rice_Image_Dataset\Basmati/*.*")
Ipsala = glob.glob(r"C:\Users\HP\Desktop\data sets\Rice_Image_Dataset\Ipsala/*.*")
Jasmine = glob.glob(r"C:\Users\HP\Desktop\data sets\Rice_Image_Dataset\Jasmine/*.*")
Karacadag = glob.glob(r"C:\Users\HP\Desktop\data sets\Rice_Image_Dataset\Karacadag/*.*")

## creating the folders (TRAIN,VALIDATION,TEST)

## CREATING THE SUB-FOLDERS FOR EACH CLASS

In [5]:
os.chdir(r"C:\Users\HP\Desktop\data sets\Rice_Image_Dataset")
t = os.listdir()
#os.chdir(r'C:\Users\HP\Desktop\xx')
#p=os.listdir()
#for i in p:
 #   os.chdir(r'C:\Users\HP\Desktop\xx\\'+i)
  #  for j in t:
   #     os.mkdir(r'C:\Users\HP\Desktop\xx\\'+i+'\\'+j)

## splitting the data

## for train set

In [6]:
z1=[]
for i in t[:5]:
    x1=rd.sample(range(len(locals()[i])),k=10000)
    z=0
    for j in x1:
        img=cv2.imread(locals()[i][j])
        cv2.imwrite(r"C:\Users\HP\Desktop\xx\train\\"+i+"\\"+i+"{}.jpg".format(z),img)
        z=z+1
        z1.append(locals()[i][j])

* to convert strings in to dictionary(key:values)-- locals()

In [7]:
for i in t[:5]:
    locals()[i]=list(set(locals()[i]).difference(z1))
    print(len(locals()[i]))

5000
5000
5000
5000
5000


In [8]:
len(z1)

50000

## FOR TEST

In [9]:
z2=[]
for i in t[:5]:
    x1=rd.sample(range(len(locals()[i])),k=2500)
    z=0
    for j in x1:
        img=cv2.imread(locals()[i][j])
        cv2.imwrite(r"C:\Users\HP\Desktop\xx\test\\"+i+"\\"+i+"{}.jpg".format(z),img)
        z=z+1
        z2.append(locals()[i][j])

In [10]:
for i in t[:5]:
    locals()[i]=list(set(locals()[i]).difference(z2))
    print(len(locals()[i]))

2500
2500
2500
2500
2500


In [11]:
len(z2)

12500

## FOR VALIDATION

In [12]:
z3=[]
for i in t[:5]:
    x1=rd.sample(range(len(locals()[i])),k=2500)
    z=0
    for j in x1:
        img=cv2.imread(locals()[i][j])
        cv2.imwrite(r"C:\Users\HP\Desktop\xx\validation\\"+i+"\\"+i+"{}.jpg".format(z),img)
        z=z+1
        z3.append(locals()[i][j])

In [13]:
for i in t[:5]:
    locals()[i]=list(set(locals()[i]).difference(z3))
    print(len(locals()[i]))

0
0
0
0
0


In [14]:
len(z3)

12500

## IMPORTING LIBRARIES

In [15]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,BatchNormalization,Flatten,Conv2D,AveragePooling2D,MaxPooling2D,Dropout

## CREATING DATA AGUMENTATION, DATA GENERATOR

In [16]:
train_data = ImageDataGenerator(
rescale=1./255,
rotation_range=20,
shear_range=0.2,
zoom_range=0.2)
train_gen = train_data.flow_from_directory(r"C:\Users\HP\Desktop\xx\train",
target_size=(200,200),
batch_size=16,
class_mode="categorical")

val_data = ImageDataGenerator(rescale=1./255)

val_gen = val_data.flow_from_directory(r"C:\Users\HP\Desktop\xx\validation",
target_size=(200,200),
batch_size=16,
class_mode="categorical")

Found 50000 images belonging to 5 classes.
Found 12500 images belonging to 5 classes.


## CREATING THE ARCHITECHTURE TRANSFER LEARNING CASE:2

In [24]:
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Flatten,Dense
from keras.callbacks import EarlyStopping as ES

In [18]:
main = VGG16(include_top=False,weights='imagenet',input_shape=(200, 200, 3))

In [19]:
main.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200, 200, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 200, 200, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 200, 200, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 100, 100, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 100, 100, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 100, 100, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 50, 50, 128)       0     

In [26]:
model = Sequential()
model.add(main)
model.add(Flatten())
model.add(Dense(80,activation="relu"))
model.add(Dense(5,activation="softmax"))

In [27]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 6, 6, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 18432)             0         
                                                                 
 dense_2 (Dense)             (None, 80)                1474640   
                                                                 
 dense_3 (Dense)             (None, 5)                 405       
                                                                 
Total params: 16,189,733
Trainable params: 16,189,733
Non-trainable params: 0
_________________________________________________________________


## COMPILING THE MODEL

In [28]:
model.compile(optimizer="rmsprop",loss="categorical_crossentropy")

In [ ]:
history = model.fit(train_gen,
                   validation_data=val_gen,
                   epochs=3,
                   steps_per_epoch=50000/16,
                   validation_steps=12500/16,
                   callbacks=[ES(monitor="val_loss", patience=5)])

## CREATING THE OUR OWN ARCHITECHTURE

In [32]:
model = Sequential()

model.add(Conv2D(20,(3,3),padding="valid",activation="relu",input_shape=(200,200,3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(10,(5,5),padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(60,(5,5),activation="relu"))

model.add(Flatten())

model.add(Dense(60,activation="relu"))

model.add(Dense(5,activation="softmax"))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 198, 198, 20)      560       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 99, 99, 20)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 99, 99, 10)        5010      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 49, 49, 10)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 45, 45, 60)        15060     
                                                                 
 flatten_2 (Flatten)         (None, 121500)           

## COMPILING THE MODEL

In [33]:
model.compile(optimizer="Adam",loss="categorical_crossentropy",metrics=["accuracy"])

## TRAINING THE MODEL WITH TRAING DATA GENERATOR

In [34]:
model.fit_generator(train_gen,steps_per_epoch=50000//16,
epochs=3,
validation_data=val_gen,validation_steps=2500//16)

C:\Users\HP\AppData\Local\Temp\ipykernel_4608\742987723.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_gen,steps_per_epoch=50000//16,


Epoch 1/3
3125/3125 [==============================] - 15882s 5s/step - loss: 0.2491 - accuracy: 0.9008 - val_loss: 3.0103 - val_accuracy: 0.6522
Epoch 2/3
3125/3125 [==============================] - 43467s 14s/step - loss: 0.0652 - accuracy: 0.9774 - val_loss: 1.0169 - val_accuracy: 0.8333
Epoch 3/3
3125/3125 [==============================] - 2764s 884ms/step - loss: 0.0546 - accuracy: 0.9818 - val_loss: 0.7246 - val_accuracy: 0.8658


## EVALUATING THE MODEL

## AT TRAINING TIME

In [40]:
tran_loss, train_acc = model.evaluate(train_gen)
print("train accuracy", train_acc)

3125/3125 [==============================] - 988s 316ms/step - loss: 0.0275 - accuracy: 0.9908
train accuracy 0.9907799959182739


* WITH TRAIN DATA i HAVE GOT 99% ACCURACY

## AT TEST TIME

In [36]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [37]:
test_generator = test_datagen.flow_from_directory(r"C:\Users\HP\Desktop\xx\test", target_size=(200, 200), batch_size=16, class_mode='categorical')

Found 12500 images belonging to 5 classes.


In [39]:
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)

782/782 [==============================] - 140s 179ms/step - loss: 0.7216 - accuracy: 0.8645
Test accuracy: 0.8644800186157227


* WITH TEST DATA (UNSEEN DATA) I HAVE GOT 86% ACCURACY